# Evaluation for Extrapolating the Learning Curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

%matplotlib inline

results_file = os.path.expanduser('data/extrapolation.csv')
df = pd.read_csv(results_file)

classifiers = df['classifier'].unique()

In [ ]:
fig_cor, axes_cor  = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'hspace': 0.4})
fig_acc, axes_acc  = plt.subplots(2, 2, figsize=(12,6), gridspec_kw={'hspace': 0.4})


for classifier in ["LinearSVC()", "GradientBoostingClassifier()", "LinearDiscriminantAnalysis()", "QuadraticDiscriminantAnalysis()", "RandomForestClassifier()"]:
    fig_ind, (ax1, ax2, ax3)  = plt.subplots(1, 3, figsize=(16,4))
    
    result_clf = df.loc[df['classifier'] == classifier]
    
    # for regression acc plot
    perf = np.abs(result_clf['performance_next_point'] - result_clf['performance_prediction']).values
    count, bins_count = np.histogram(perf, bins=1000)
    pdf = count / sum(count)
    cdf = np.cumsum(pdf)
    
    # for curve plot
    curve_plot_x = np.arange(0, 1, 0.01)
    curve_plot_y = []
    for i in curve_plot_x:
        cur_val = ((result_clf['delta_current_actual'] < i) & (result_clf['delta_current_prediction'].values < i)) | ((result_clf['delta_current_actual'].values >= i) & (result_clf['delta_current_prediction'].values >= i))
        curve_plot_y.append(sum(cur_val) / len(cur_val))
    
    # build upon individual plots
    fig_ind.suptitle(classifier)
    ax1.scatter(result_clf['delta_current_actual'], result_clf['delta_current_prediction'])
    ax2.plot(bins_count[1:], cdf)
    ax3.plot(curve_plot_x, curve_plot_y)
    ax3.set_ylim([0,1.05])
    ax3.axhline(0.9, linestyle="--", linewidth=1, color="black")
    
    # adds to general plot
    axes_cor[0].scatter(result_clf['delta_current_prediction'], result_clf['delta_current_actual'], s=3)
    axes_cor[0].set_title("Actual vs. predicted improvements per learner")
    axes_acc[0][0].plot(bins_count[1:], cdf, label=classifier[:-2])
    axes_acc[0][0].set_title("Correct classifications by tolerance per learner")
    axes_acc[1][0].plot(curve_plot_x, curve_plot_y)
    axes_acc[1][0].set_title("Correct classifications by threshold per learner")

### now for the portfolio selection (based on FULL portfolio)
best_actual = []
best_predicted = []
for dataset, df_dataset in df.groupby("task_id"):
    # best performing classifier at halfway point (lower is better)
    best_score_on_half = min(df_dataset["performance_curve_end"])
    # best performing classifier at full point (lower is better)
    best_score_on_full = min(df_dataset['performance_next_point'])
    # delta of the best obtained score at full point minus halfway point (higher is better!)
    delta_best = best_score_on_half - best_score_on_full
    # per-classifier prediction how much it will improve over the best classifier so far (higher is better)
    df_dataset['delta_currentbest_prediction'] = best_score_on_half - df_dataset["performance_prediction"]
    # the best predicted performance gain (maximize)
    best_predicted_delta = max(df_dataset["delta_currentbest_prediction"])
    
    best_actual.append(delta_best)
    best_predicted.append(best_predicted_delta)
best_actual = np.array(best_actual)
best_predicted = np.array(best_predicted)

# for regression acc plot
perf = np.abs(best_actual - best_predicted)
count, bins_count = np.histogram(perf, bins=1000)
pdf = count / sum(count)
cdf = np.cumsum(pdf)

# for curve plot
curve_plot_x = np.arange(0, 1, 0.01)
curve_plot_y = []
for i in curve_plot_x:
    cur_val = ((best_actual < i) & (best_predicted < i)) | ((best_actual >= i) & (best_predicted >= i))
    curve_plot_y.append(sum(cur_val) / len(cur_val)) 

portfolio_color = "C5"
axes_cor[1].scatter(best_predicted, best_actual, s=8, color=portfolio_color)
axes_cor[1].set_title("Actual vs. predicted improvements for portfolio")
axes_acc[0][1].plot(bins_count[1:], cdf, color=portfolio_color, label="Portfolio")
axes_acc[0][1].set_title("Correct classifications by tolerance for portfolio")
axes_acc[1][1].plot(curve_plot_x, curve_plot_y, color=portfolio_color)
axes_acc[1][1].set_title("Correct classifications by threshold for portfolio")
    
for col in range(2):
    axes_cor[col].set_xlim([-0.2, 0.6])
    axes_cor[col].set_ylim([-0.2, 0.6])
    axes_cor[col].plot([-0.2, 1], [-0.2, 1], linestyle="--", linewidth=1, color="black")
    
    axes_acc[1][col].set_xlim([0, 0.34])
    axes_acc[1][col].set_ylim([0.5,1.05])
    
    axes_cor[col].set_xlabel("Predicted Improvement")
    axes_cor[col].set_ylabel("Actual Improvement")
    axes_acc[0][col].set_xlabel("Tolerated Deviation for Correct Prediction")
    axes_acc[0][col].axhline(0.9, linestyle="--", linewidth=1, color="black")
    axes_acc[0][col].set_ylabel("Accuracy")
    axes_acc[1][col].set_xlabel("Required Improvement")
    axes_acc[1][col].set_ylabel("Accuracy")
    axes_acc[1][col].axhline(0.95, linestyle="--", linewidth=1, color="black")

handles, labels = axes_cor[0].get_legend_handles_labels()
handles_port, labels_port = axes_cor[1].get_legend_handles_labels()
handles.extend(handles_port)
labels.extend(labels_port)
lgd = fig_cor.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.45, 0.06), ncol=3)
#fig.tight_layout()
fig_cor.savefig("plots/results-correlation.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
fig_acc.savefig("plots/results-accuracy.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()
# print('results on %d tasks:' % len(result_clf['task_id']), result_clf['task_id'].values)